In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/computer-science-theory-qa-dataset/intents.json
/kaggle/input/data-science-glossary-for-qa/glossary1.json


A. Basic Q & A system using Keyboard Matching


In [8]:
import json
import re

# Step 1: Load the dataset
def load_qa_data(file_path):
    try:
        with open(file_path, 'r') as f:
            qa_data = json.load(f)
        return qa_data
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return {}

# Step 2: Clean and preprocess text (optional stop word removal)
def clean_text(text):
    stop_words = {'the', 'is', 'in', 'at', 'which', 'on', 'and', 'a', 'an'}
    # Remove non-alphabet characters and split into words
    words = re.sub(r'[^a-zA-Z\s]', '', text.lower()).split()
    # Remove stop words
    return [word for word in words if word not in stop_words]

# Step 3: Keyword matching function
def find_answer(question, qa_data):
    cleaned_question = clean_text(question)
    
    for q, a in qa_data.items():
        cleaned_q = clean_text(q)
        if all(keyword in cleaned_q for keyword in cleaned_question):
            return a

    return "Sorry, I don't know the answer to that question."

# Step 4: Main function
def main():
    file_path = '/kaggle/input/data-science-glossary-for-qa/glossary1.json'
    qa_data = load_qa_data(file_path)  # Load the Q&A dataset

    if not qa_data:
        return  # Exit if dataset could not be loaded

    while True:
        user_question = input("Ask a question (or type 'exit' to quit): ")
        if user_question.lower() in ['exit', 'quit']:
            print("Exiting the Q&A system. Goodbye!")
            break

        answer = find_answer(user_question, qa_data)
        print("Answer:", answer)

if __name__ == "__main__":
    main()


Ask a question (or type 'exit' to quit):  tell about data science 


Answer: Sorry, I don't know the answer to that question.


Ask a question (or type 'exit' to quit):  exit 


Answer: Sorry, I don't know the answer to that question.


Ask a question (or type 'exit' to quit):  quit


Exiting the Q&A system. Goodbye!


B.Building a Q & A system with BERT 

In [ ]:
# Import required libraries
from datasets import load_dataset
from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments
import torch

# Load the dataset
dataset = load_dataset('squad')

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocessing function
def preprocess_data(examples):
    inputs = tokenizer(
        examples['context'], 
        examples['question'], 
        truncation=True, 
        padding='max_length'
    )

    start_positions = []
    end_positions = []

    # Assuming 'examples['answers']' is a list of dictionaries
    for answer in examples['answers']:
        start_positions.append(answer['answer_start'][0])  # Assuming single answer per example
        end_positions.append(answer['answer_start'][0] + len(answer['text'][0]))

    inputs.update({
        'start_positions': start_positions,
        'end_positions': end_positions
    })
    
    return inputs


# Map the preprocessing function to the dataset
train_dataset = dataset['train'].map(preprocess_data, batched=True)

# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

# Step 5: Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Load the BERT model for question answering
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dataset['validation'],
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

# Example: Predict on a single question-context pair
question = "What is the capital of France?"
context = "The capital of France is Paris."

# Tokenize the input example
inputs = tokenizer(question, context, return_tensors='pt', padding=True, truncation=True)

# Get model prediction
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

# Get the predicted start and end positions
predicted_start = torch.argmax(start_logits)
predicted_end = torch.argmax(end_logits)

# Get the answer from the context
answer = context[predicted_start.item():predicted_end.item() + 1]
print(f"Predicted Answer: {answer}")


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


C. Question Answering on SQUAD dataset using transformers 

In [15]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer, Trainer, TrainingArguments

# Step 1: Load the dataset from CSV using pandas
train_df = pd.read_csv('/kaggle/input/squad-20-csv-file/squad_csv/train-squad.csv')
validation_df = pd.read_csv('/kaggle/input/squad-20-csv-file/squad_csv/validation-squad.csv')

# Convert pandas DataFrames to Huggingface Datasets
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)

# Step 2: Initialize the pre-trained model and tokenizer
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Check if a GPU is available and move the model to GPU if it is
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Step 3: Preprocessing function
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = examples["context"]

    # Tokenizing questions and context
    inputs = tokenizer(
        questions, contexts,
        max_length=384,
        truncation="only_second",
        padding="max_length"
    )

    # Using the "answer_start" and "text" from the dataset to create labels
    start_positions = examples["answer_start"]
    end_positions = []

    for start, answer in zip(examples["answer_start"], examples["text"]):
        if answer is not None:
            end_positions.append(start + len(answer))
        else:
            end_positions.append(start)  # Default to start if answer is None

    # Add start and end positions to the inputs
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

# Step 4: Preprocess the dataset
train_dataset = train_dataset.map(preprocess_function, batched=True)
validation_dataset = validation_dataset.map(preprocess_function, batched=True)

# Step 5: Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01
)

# Step 6: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset
)

# Step 7: Fine-tune the model
trainer.train()

# Step 8: Inference (Sample Input)
question = "Who developed BERT?"
context = "BERT was developed by researchers at Google."

# Tokenize the input question and context
inputs = tokenizer(question, context, return_tensors='pt').to(device)  # Move inputs to the same device as the model

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get start and end logits
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# Find the start and end positions of the answer
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores) + 1

# Convert tokens back to the answer string
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

# Display the result
print("Question:", question)
print("Answer:", answer)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/86821 [00:00<?, ? examples/s]

Map:   0%|          | 0/10388 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,5.303800,nan
2,5.060900,nan


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

Question: Who developed BERT?
Answer: [CLS] who developed bert ? [SEP] bert was developed by researchers at google . [SEP]
